In [1]:
import tensorflow as tf
import csv
import math
from tensorflow import keras



# Helper libraries
import numpy as np

import rospy
import os
import json
import numpy as np
import random
import time
import sys
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Activation
from collections import deque

In [2]:
batch_size = 2
train_start = 64
memory = deque(maxlen=1000000)


In [3]:

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# Note that we can name any layer by passing it a "name" argument.
state = keras.layers.Input(shape=(14,), name='state')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
d1 = Dense(512, activation='relu')(state)
#auxiliary_output = keras.layers.Dense(1, activation='sigmoid', name='aux_output')(d1)

action = keras.layers.Input(shape=(2,), name='action')
x = keras.layers.concatenate([d1, action])

x = Dense(512, activation='relu')(x)
x = Dense(512, activation='relu')(x)


# And finally we add the main logistic regression layer
q_val= Dense(1, activation='sigmoid', name='main_output')(x)

critic = Model(inputs=[state, action], outputs=[q_val])
#critic.summary()

In [4]:


state = keras.layers.Input(shape=(14,), name='main_input')

# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
x = Dense(512, activation='relu')(state)
x = Dense(512, activation='relu')(state)
x = Dense(512, activation='relu')(state)
#auxiliary_output = keras.layers.Dense(1, activation='sigmoid', name='aux_output')(d1)

lin_vel = Dense(1, activation='sigmoid', name='lin_vel')(x)
ang_vel = Dense(1, activation='tanh', name='ang_vel')(x)
output = keras.layers.concatenate([lin_vel, ang_vel])

actor = Model(inputs=[state], outputs=output)
#actor.summary()


In [5]:
import numpy as np

In [6]:
critic.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mse',
              metrics=['accuracy'])

actor.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mse',
              metrics=['accuracy'])


In [7]:
#model.fit(ml_in, ml_out, epochs=10)
test = np.ones(14)
test2 = np.ones(2)


In [8]:
#predictions = model.predict(ml_in)
blank = actor.predict(test.reshape(1, len(test)))
blank2 = critic.predict([test.reshape(1, len(test)), test2.reshape(1,len(test2))])

In [ ]:
#predictions[0:20]
#predictions.shape
past_action = np.zeros((1,2))
print blank.shape
past_action.shape

In [71]:
t = np.empty([0,14])
t2 = np.empty([0,2])
test = np.ones(14)
test2 = np.ones(2)
test = test.reshape(1, len(test))
test2 = test2.reshape(1, len(test2))
print test2.shape
print t2.shape
t = np.append(t , test, axis = 0)
t2 = np.append(t2 , test2, axis = 0)

print test.shape
print test2.shape

(1, 2)
(0, 2)
(1, 14)
(1, 2)


In [72]:
blank2 = critic.predict([test, test2])


In [70]:
test[0][2] = 3
print test.shape
print test2.shape

TypeError: 'numpy.float64' object does not support item assignment

In [ ]:
#sess = tf.Session()
action = tf.placeholder(tf.float32, [None, 2])
action_2 = tf.placeholder(tf.float32, [None, 2])
state = tf.placeholder(tf.float64, [None, 14])

#state_1 = tf.placeholder(tf.float32, [None, 14])
action_grads = tf.gradients(critic.output, action) #GRADIENTS for policy update
#sess.run(tf.global_variables_initializer())






In [ ]:
#sess = tf.Session()
#sess.run(tf.global_variables_initializer())

def grads (model, states, actions):
    grad = action_grads, feed_dict={
            state: states,
            action: actions
    }
    return grad

grads(critic, test, test2)

In [ ]:
#critic.get_weights()
with tf.Session() as sess:
    print sess.run(action_grads,feed_dict={
    state: test, 
    action: test2})
#critic.get_weights()
#print temp
#sess.close()

In [ ]:
import tensorflow as tf

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a+b

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print sess.run(c,feed_dict={a:1.0, b:1.0})
sess.close()
    

In [ ]:

a = tf.constant(0.)
b = 2 * a
g = tf.gradients(a + b, [a, b], stop_gradients=[a, b])
print g

In [ ]:
actor.output_shape

In [ ]:
memory.append((test, test2))

In [35]:
len(memory)

1

In [10]:
test3 = np.asarray([ 2.76205611,  2.06204796,  1.59506261,  1.24448633,  1.11614728,
        1.05906844,  1.12388086,  1.14259541,  1.03247726,  1.01678181,
        0.        ,  0.        , -2.42      ,  2.35      ])

In [39]:
memory.append((test, test2, test3, blank))

In [36]:
t = np.asarray(memory[0])
t2 = np.asarray(memory[2])
t3 = np.asarray(memory[3])

IndexError: deque index out of range

In [23]:
t3 = t3.reshape(1, len(t3))

In [59]:
memory[1][1]

array([ 1.,  1.])

In [48]:
l = np.empty(0)

In [52]:
states = np.empty(([0,14]), dtype=np.float64)
print states.shape
print np.asarray(memory[1][2]).shape
states = np.append(states, np.asarray(memory[1][2]).reshape(1, 14), axis = 0)

(0, 14)
(14,)


In [75]:
blank2 = np.append(blank2, blank2, axis = 0)
print blank2.shape
test2 = np.append(test2, test2, axis = 0)
print test2.shape
test = np.append(test, test, axis = 0)
print test.shape


(8, 1)
(8, 2)
(8, 14)


In [60]:
t = []
t.append(True)
t.append(False)

In [76]:
loss = 0

In [81]:
tes= critic.train_on_batch([test, test2], blank2) 

In [82]:
tes

[0.0, 0.0]

In [7]:
class OrnsteinUhlenbeckActionNoise:
    def __init__(self, mu, sigma=0.3, theta=.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()

    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + \
                self.sigma * np.sqrt(self.dt) * np.random.normal(size=self.mu.shape)
        x[0] = abs(x[0])
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

    def __repr__(self):
        return 'OrnsteinUhlenbeckActionNoise(mu={}, sigma={})'.format(self.mu, self.sigma)

In [8]:
actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(2))

In [14]:
for x in range(200):
    print actor_noise()

[ 0.01117956 -0.00213796]
[ 0.03669061  0.0362878 ]
[ 0.09078902 -0.00827449]
[ 0.06801484 -0.03093242]
[ 0.0918488  -0.02359712]
[ 0.08019879 -0.00834973]
[ 0.05865581  0.01688653]
[ 0.06244136  0.02763384]
[ 0.0532865  -0.00797641]
[ 0.03027308 -0.0087905 ]
[ 0.06696278 -0.0104088 ]
[ 0.07233983  0.02573834]
[ 0.1087147   0.07257342]
[ 0.1057209   0.10663628]
[ 0.08666155  0.09591483]
[ 0.08702894  0.1394619 ]
[ 0.02228508  0.13172905]
[ 0.02407698  0.09392548]
[ 0.00753071  0.07152991]
[ 0.05458295  0.09763194]
[ 0.0060717   0.05389203]
[ 0.04942431  0.12067119]
[ 0.09850357  0.09868975]
[ 0.04559672  0.13858909]
[ 0.05318048  0.1422861 ]
[ 0.03989534  0.20423657]
[ 0.00592788  0.19442798]
[ 0.00841472  0.18474455]
[ 0.04278187  0.19853825]
[ 0.01436192  0.21321828]
[ 0.02867564  0.24207964]
[ 0.02689482  0.26935606]
[ 0.01414772  0.25847983]
[ 0.04423991  0.20396005]
[ 0.09719682  0.1561384 ]
[ 0.12900001  0.18199144]
[ 0.08114437  0.18344139]
[ 0.07916776  0.15815134]
[ 0.07626911

In [6]:
actor_noise()/10

array([ 0.00387038,  0.00550279])